In [13]:
from requests import Request, Session
import json
import numpy as np
import pandas as pd

In [14]:
import time

In [15]:
# url = 'https://cloud.iexapis.com/stable/stock/AAPL/book'
url_book = 'https://cloud.iexapis.com/stable/deep/book'
url_trade = 'https://cloud.iexapis.com/stable/deep/trades'
# url = 'https://sandbox.iexapis.com/stable/deep/book'

In [16]:
parameters_book = {
#     'token': 'Tsk_83f2c0eee6bb4600b8d8f9936a5769f2',
    'token': 'sk_b8918e562940417ca9b6615a914b75bd',
    'symbols': "AAPL"
}
parameters_trade = {
    'token': 'sk_b8918e562940417ca9b6615a914b75bd',
    'symbols': 'AAPL',
    'last': 500
}
headers = {
    'Accepts': 'application/json'
}

In [17]:
session = Session()
session.headers.update(headers)

# Function

In [18]:
def crawl_book():
    columns = ['price','size','timestamp']
    bids = pd.DataFrame(columns=columns)
    asks = pd.DataFrame(columns=columns)
    response = session.get(url_book, params=parameters_book)
    if not response.status_code == 200:
        print(response.text)
        return bids, asks
    data = json.loads(response.text)
    if not 'AAPL' in data:
        return bids, asks
    else:
        data = data.get('AAPL')
    if 'bids' in data and 'asks' in data:
        bids = pd.DataFrame(data.get('bids'))
        asks = pd.DataFrame(data.get('asks')) 
    return bids, asks

In [19]:
def crawl_trade():
    columns = ['price','size','timestamp']
    trades = pd.DataFrame(columns=columns)
    response = session.get(url_trade, params=parameters_trade)
    if not response.status_code == 200:
        print(response.text)
        return trades
    data = json.loads(response.text)
    if not 'AAPL' in data:
        return trades
    else:
        data = pd.DataFrame(data.get('AAPL'))
    if not data.empty:
        trades = data[['price','size','timestamp']]
    return trades

# Collect data

In [20]:
total_trades = pd.read_csv('trades.csv')

## New data

In [31]:
i = 0
bids = pd.DataFrame(columns=['price','size','timestamp'])
asks = pd.DataFrame(columns=['price','size','timestamp'])
old_trades = pd.DataFrame(columns=['price','size','timestamp'])
trades = pd.DataFrame(columns=['price','size','timestamp'])
while i >= 0 and i < 50:
    new_bids, new_asks = crawl_book()
    new_trades = crawl_trade()
    cur_time = round(time.time())
    if not new_trades.empty and not new_trades.equals(old_trades):
        trades = trades.append(new_trades)
        old_trades = new_trades
    if not new_bids.empty and not new_bids.equals(bids):
        new_bids.to_csv('bids/{0}.csv'.format(cur_time), index=False)
        bids = new_bids
    if not new_asks.empty and not new_asks.equals(asks):
        new_asks.to_csv('asks/{0}.csv'.format(cur_time), index=False)
        asks = new_asks
    i = i + 1
    if i % 10 == 0: # mod 100
        if not len(trades.merge(total_trades)) == len(trades):
            total_trades = total_trades.append(trades)
            total_trades.to_csv('trades.csv', index=False)
        trades = pd.DataFrame(columns=['price','size','timestamp'])
    time.sleep(1)